In [ ]:
# Computer vision/graphics library
import cv2

# Gif writer
import imageio

# Display libraries
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# Calculations and Deep Learning library
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
! pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [ ]:
cyan = (255, 255, 0)
magenta = (255, 0, 255)

In [ ]:
EDGE_COLORS = {
    (0, 1): magenta,
    (0, 2): cyan,
    (1, 3): magenta,
    (2, 4): cyan,
    (0, 5): magenta,
    (0, 6): cyan,
    (5, 7): magenta,
    (7, 9): cyan,
    (6, 8): magenta,
    (8, 10): cyan,
    (5, 6): magenta,
    (5, 11): cyan,
    (6, 12): magenta,
    (11, 12): cyan,
    (11, 13): magenta,
    (13, 15): cyan,
    (12, 14): magenta,
    (14, 16): cyan
}

In [ ]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/1")
movenet = model.signatures["serving_default"]

Adjust the input size
According to the official documentation :

The height/width are both multiple of 32.
The height to width ratio is close (and enough) to cover the original image's aspect ratio.
Make the larger side to be 256 (one should adjust this based on the speed/accuracy requirements). For example, a 720p image (i.e. 720x1280 (HxW)) should be resized and padded to 160x256 image.
Following these rules, our input_gif would have been reshaped like this :

width : 461 ---> 256 (the larger side = 256).
height : 250 ---> 250 * (250/461) ~136. Since the rule 1 specifies the height to be a mutliple of 32, we'd have to round it to the closest one, which is 128.
*Note : 250/461 is the aspect ratio.

For visualization purposes, we'll set the input_size to 256*256.

In [ ]:
#initial_width, initial_height = (640,360)
WIDTH = 256
HEIGHT = 256


In [ ]:
def loop(frame, keypoints, threshold=0.1):
    """
    Main loop : Draws the keypoints and edges for each instance
    """

    # Loop through the results
    for instance in keypoints:
        # Draw the keypoints and get the denormalized coordinates
        denormalized_coordinates = draw_keypoints(frame, instance, threshold)
        # Draw the edges
        draw_edges(denormalized_coordinates, frame, EDGE_COLORS, threshold)

In [ ]:
def draw_keypoints(frame, keypoints, threshold=0.1):
    """Draws the keypoints on a image frame"""

    # Denormalize the coordinates : multiply the normalized coordinates by the input_size(width,height)
    denormalized_coordinates = np.squeeze(np.multiply(keypoints, [WIDTH,HEIGHT,1]))
    #Iterate through the points
    for keypoint in denormalized_coordinates:
        # Unpack the keypoint values : y, x, confidence score
        keypoint_y, keypoint_x, keypoint_confidence = keypoint
        if keypoint_confidence > threshold:
            """"
            Draw the circle
            Note : A thickness of -1 px will fill the circle shape by the specified color.
            """
            cv2.circle(
                img=frame,
                center=(int(keypoint_x), int(keypoint_y)),
                radius=4,
                color=(0,255,0),
                thickness=-1
            )
    return denormalized_coordinates

In [ ]:
def draw_edges(denormalized_coordinates, frame, edges_colors, threshold=0.1):
    """
    Draws the edges on a image frame
    """

    # Iterate through the edges
    for edge, color in edges_colors.items():
        # Get the dict value associated to the actual edge
        p1, p2 = edge
        # Get the points
        y1, x1, confidence_1 = denormalized_coordinates[p1]
        y2, x2, confidence_2 = denormalized_coordinates[p2]
        # Draw the line from point 1 to point 2, the confidence > threshold
        if (confidence_1 > threshold) & (confidence_2 > threshold):
            cv2.line(
                img=frame,
                pt1=(int(x1), int(y1)),
                pt2=(int(x2), int(y2)),
                color=color,
                thickness=1,
                lineType=cv2.LINE_AA # Gives anti-aliased (smoothed) line which looks great for curves
            )

In [ ]:
def progress(value, max=100):
    """
    Returns an HTML progress bar with a certain value. Used within each step
    """


    return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value,
                max=max))

In [ ]:
def load_gif():
    """
    Loads the gif and return its details
    """

    # Load the gif
    gif = cv2.VideoCapture("/content/Splitted.mp4")
    # Get the frame count
    frame_count = int(gif.get(cv2.CAP_PROP_FRAME_COUNT))
    # Display parameter
    print(f"Frame count: {frame_count}")

    """""
    Initialize the video writer
    We'll append each frame and its drawing to a vector, then stack all the frames to obtain a sequence (video).
    """
    output_frames = []

    # Get the initial shape (width, height)
    initial_shape = []
    initial_shape.append(int(gif.get(cv2.CAP_PROP_FRAME_WIDTH)))
    initial_shape.append(int(gif.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    return gif, frame_count, output_frames, initial_shape

In [ ]:
import json

def run_inference():
    keypoints_list = []

    """
    Runs inferences then starts the main loop for each frame
    """

    # Load the gif
    gif, frame_count, output_frames, initial_shape = load_gif()
    # Set the progress bar to 0. It ranges from the first to the last frame
    bar = display(progress(0, frame_count-1), display_id=True)

    # Loop while the gif is opened
    while gif.isOpened():

        # Capture the frame
        ret, frame = gif.read()

        # Exit if the frame is empty
        if frame is None:
            break

        # Retrieve the frame index
        current_index = gif.get(cv2.CAP_PROP_POS_FRAMES)

        # Copy the frame
        image = frame.copy()
        image = cv2.resize(image, (WIDTH,HEIGHT))
        # Resize to the target shape and cast to an int32 vector
        input_image = tf.cast(tf.image.resize_with_pad(image, WIDTH, HEIGHT), dtype=tf.int32)
        # Create a batch (input tensor)
        input_image = tf.expand_dims(input_image, axis=0)

        # Perform inference
        results = movenet(input_image)
        """
        Output shape :  [1, 6, 56] ---> (batch size), (instances), (xy keypoints coordinates and score from [0:50]
        and [ymin, xmin, ymax, xmax, score] for the remaining elements)
        First, let's resize it to a more convenient shape, following this logic :
        - First channel ---> each instance
        - Second channel ---> 17 keypoints for each instance
        - The 51st values of the last channel ----> the confidence score.
        Thus, the Tensor is reshaped without losing important information.
        """

        keypoints = results["output_0"].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_list.append(keypoints.tolist())


        # Loop through the results
        loop(image, keypoints, threshold=0.3)

        # Get the output frame : reshape to the original size
        frame_rgb = cv2.cvtColor(
            cv2.resize(
                image,(initial_shape[0], initial_shape[1]),
                interpolation=cv2.INTER_LANCZOS4
            ),
            cv2.COLOR_BGR2RGB # OpenCV processes BGR images instead of RGB
        )

        # Add the drawings to the output frames
        output_frames.append(frame_rgb)

        # Update the progress bar
        bar.update(progress(current_index, frame_count-1))

    keypoints_json = json.dumps(keypoints_list)

    # Release the object
    gif.release()

    with open('keypoints.json', 'w') as file:
      file.write(keypoints_json)


    print("Completed !")

    return output_frames

In [ ]:
output_frames = run_inference()

In [ ]:
from tensorflow_docs.vis import embed

In [ ]:
# Stack the output frames horizontally to compose a sequence
output = np.stack(output_frames, axis=0)
# Write the sequence to a gif
imageio.mimsave("./output.mp4", output, fps=29.97)
# Embed the output to the notebook
embed.embed_file("./output.mp4")